Download data from openDAP to compute DOC flux

In [3]:
"""Import packages"""
import datetime
import xarray as xr
import numpy as np
import scipy.io as sio

In [4]:
"""Get openDAP URLs and download data"""
url1='http://tds.marine.rutgers.edu/thredds/dodsC/roms/ecb/del/2007-2008/avg'
url2='http://tds.marine.rutgers.edu/thredds/dodsC/roms/ecb/del/2009-2011/avg'
ds1 = xr.open_dataset(url1)
ds2 = xr.open_dataset(url2)

# slicing throught the time, xi_u and eta_u dimensions
Hvom1 = ds1.sel(ocean_time=slice('2007','2008'),eta_v=126,xi_v=slice(25, 54)).Hvom
Hvom_slbDOC1 =ds1.sel(ocean_time=slice('2007','2008'),eta_v=126,xi_v=slice(25, 54)).Hvom_semilabileDOC
Hvom_rfrDOC1 =ds1.sel(ocean_time=slice('2007','2008'),eta_v=126,xi_v=slice(25, 54)).Hvom_refractoryDOC
#
Hvom2 = ds2.sel(ocean_time=slice('2009','2011'),eta_v=126,xi_v=slice(25, 54)).Hvom
Hvom_slbDOC2 =ds2.sel(ocean_time=slice('2009','2011'),eta_v=126,xi_v=slice(25, 54)).Hvom_semilabileDOC
Hvom_rfrDOC2 =ds2.sel(ocean_time=slice('2009','2011'),eta_v=126,xi_v=slice(25, 54)).Hvom_refractoryDOC

In [5]:
# concatenate arrays
Hvom_DOC1=Hvom_slbDOC1 + Hvom_rfrDOC1
Hvom_DOC2=Hvom_slbDOC2 + Hvom_rfrDOC2
Hvom = xr.concat([Hvom1, Hvom2], dim='ocean_time')
Hvom_DOC = xr.concat([Hvom_DOC1, Hvom_DOC2], dim='ocean_time')

In [6]:
"""Daily Huon*DOC - Following Aboozar's suggestion, as Pierre did:
tracers (e.g DOC, salt) are calculated at the middle of each cell but
velocities are calculated at the sides, so the right way to calculate
uhvom x DOC would be to average DOC at 2 center points before and after
the side. I took cell j=126 for tracers and so I'm averaging DOC in the
center of cell 125 and 126 """
# 2007-2008 run
DOCslb1 =ds1.sel(ocean_time=slice('2007','2008'),eta_rho=125,xi_rho=slice(25, 54)).semilabileDOC
DOCslb2 =ds1.sel(ocean_time=slice('2007','2008'),eta_rho=126,xi_rho=slice(25, 54)).semilabileDOC
DOCrfr1 =ds1.sel(ocean_time=slice('2007','2008'),eta_rho=125,xi_rho=slice(25, 54)).refractoryDOC
DOCrfr2 =ds1.sel(ocean_time=slice('2007','2008'),eta_rho=126,xi_rho=slice(25, 54)).refractoryDOC
wrk1 = 0.5*(DOCslb1 + DOCslb2)
wrk2 = 0.5*(DOCrfr1 + DOCrfr2)
DOC1 = wrk1 + wrk2
# 2009-2011 run
DOCslb3 =ds2.sel(ocean_time=slice('2009','2011'),eta_rho=125,xi_rho=slice(25, 54)).semilabileDOC
DOCslb4 =ds2.sel(ocean_time=slice('2009','2011'),eta_rho=126,xi_rho=slice(25, 54)).semilabileDOC
DOCrfr3 =ds2.sel(ocean_time=slice('2009','2011'),eta_rho=125,xi_rho=slice(25, 54)).refractoryDOC
DOCrfr4 =ds2.sel(ocean_time=slice('2009','2011'),eta_rho=126,xi_rho=slice(25, 54)).refractoryDOC
wrk3 = 0.5*(DOCslb3 + DOCslb4)
wrk4 = 0.5*(DOCrfr3 + DOCrfr4)
DOC2 = wrk3 + wrk4
# concatenate arrays
DOC = xr.concat([DOC1, DOC2], dim='ocean_time')

In [12]:
salt1 = ds1.sel(ocean_time=slice('2007','2008'),eta_rho=125,xi_rho=slice(25, 54)).salt
salt2 = ds1.sel(ocean_time=slice('2007','2008'),eta_rho=126,xi_rho=slice(25, 54)).salt
salta = 0.5*(salt1 +salt2)
temp1 = ds1.sel(ocean_time=slice('2007','2008'),eta_rho=125,xi_rho=slice(25, 54)).temp
temp2 = ds1.sel(ocean_time=slice('2007','2008'),eta_rho=126,xi_rho=slice(25, 54)).temp
tempa = 0.5*(temp1 + temp2)
salt3 = ds2.sel(ocean_time=slice('2009','2011'),eta_rho=125,xi_rho=slice(25, 54)).salt
salt4 = ds2.sel(ocean_time=slice('2009','2011'),eta_rho=126,xi_rho=slice(25, 54)).salt
saltb = 0.5*(salt3 +salt4)
temp3 = ds2.sel(ocean_time=slice('2009','2011'),eta_rho=125,xi_rho=slice(25, 54)).temp
temp4 = ds2.sel(ocean_time=slice('2009','2011'),eta_rho=126,xi_rho=slice(25, 54)).temp
tempb = 0.5*(temp3 + temp4)
# concatenate arrays
salt = xr.concat([salta,saltb], dim='ocean_time')
temp = xr.concat([tempa,tempb], dim='ocean_time')

In [13]:
def make_dataset(data_array_dict):
    xr_ds = xr.Dataset(data_array_dict)
    return xr_ds

In [14]:
makeds = make_dataset(dict(Hvom=Hvom, Hvom_DOC=Hvom_DOC, DOC=DOC, salt=salt, temp=temp))
makeds.to_netcdf('delaware_bay_docflux.nc')